In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import yaml
import pickle
import pandas as pd

linewidth = 4
alpha = 0.2
figsize = (12,6)
ylim = (0, 200)
colors = plt.cm.Dark2.colors

plt.rcParams.update(
    {
        'font.size': 24,
        'text.usetex': True,
        'axes.linewidth': linewidth,
        'lines.linewidth': linewidth,
        'xtick.major.width': linewidth,
        'ytick.major.width': linewidth,
        'xtick.major.size': 2*linewidth,
        'ytick.major.size': 2*linewidth,
    }
)

# Regret Plots

In [ ]:
input_dir = "../data/logistic_feedback/ackley"
save_figs = True

algo_label = {
    "EmpiricalMean": r"$\textsc{Ind-UCB}$",
    "LogisticUCB1": r"$\textsc{Log-UCB1}$",
    "LGPUCB": r"$\textsc{LGP-UCB}(Ours)$",
    "GPRegressor": r"$\textsc{GP-UCB}$",
}
algo_color = {
    "LGPUCB": colors[0],
    "GPRegressor": colors[1],
    "EmpiricalMean": colors[2],
    "LogisticUCB1": colors[3],
}
algo_linestyle = {
    "EmpiricalMean": "dotted",
    "LogisticUCB1": "dashdot",
    "LGPUCB": "solid",
    "GPRegressor": "dashed",
}
models = [
    "EmpiricalMean",
    "LogisticUCB1",
    "LGPUCB",
    "GPRegressor",
]
config = yaml.safe_load(open(input_dir + "/config.yaml", "r"))
print("Config: ")
for key, values in config.items():
    print(key, ": ", values)

print("\nAlgorithm results paths")
results_dir = {}
for model_name in models:
    result_file = model_name + ".pkl"
    print("Result path: ", result_file)
    try:
        with open(input_dir + "/" + result_file, "rb") as f:
            result = pickle.load(f)
            results_dir[model_name] = result
    except FileNotFoundError:
        print("Results not found for: ", model_name)
        pass

def moving_average(a, n=3):
    ret = jnp.cumsum(a, dtype=float)
    ret = ret.at[n:].set(ret[n:] - ret[:-n])
    return ret[n - 1:] / n

regret_statistics = {}
fig, ax = plt.subplots(figsize=figsize)
for algo_name in models:
    result = results_dir[algo_name]
    regret = result["regret"]  # Shape: (num_seeds, num_iter)
    regret = jax.vmap(moving_average, in_axes=(0, None))(regret, 100)
    mean_regret = jnp.mean(regret, axis=0)
    std_regret = jnp.std(regret, axis=0) / jnp.sqrt(regret.shape[0])
    regret_statistics[algo_name] = (mean_regret, std_regret)
    ax.plot(
        mean_regret,
        label=algo_label[algo_name],
        color=algo_color[algo_name],
        linestyle=algo_linestyle[algo_name],
        linewidth=plt.rcParams['lines.linewidth'],
    )
    ax.fill_between(
        jnp.arange(len(mean_regret)),
        mean_regret - std_regret,
        mean_regret + std_regret,
        alpha=0.2,
        color=algo_color[algo_name]
    )
lines, legends = ax.get_legend_handles_labels()
lgd = fig.legend(lines, legends, loc="lower center", bbox_to_anchor=(0.5, -0.24), ncol=2)
plt.xlabel(r"Time Step: $T$", fontsize=30)
plt.ylabel(r"Simple Regret", fontsize=30)
if save_figs:
    plt.savefig(input_dir + "/regret.png", bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

fig, ax = plt.subplots(figsize=figsize)
for algo_name in models:
    result = results_dir[algo_name]
    regret = result["regret"]
    average_cumulative_regret = jnp.cumsum(regret, axis=-1)
    mean_regret = jnp.mean(average_cumulative_regret, axis=0)
    std_regret = jnp.std(average_cumulative_regret, axis=0) / jnp.sqrt(average_cumulative_regret.shape[0])
    ax.plot(
        mean_regret,
        label=algo_label[algo_name],
        color=algo_color[algo_name],
        linestyle=algo_linestyle[algo_name],
        linewidth=plt.rcParams['lines.linewidth'],
    )
    ax.fill_between(
        jnp.arange(len(mean_regret)),
        mean_regret - std_regret,
        mean_regret + std_regret,
        alpha=0.2,
        color=algo_color[algo_name],
    )
lines, legends = ax.get_legend_handles_labels()
lgd = fig.legend(lines, legends, loc="lower center", bbox_to_anchor=(0.5, -0.24), ncol=2)
plt.xlabel(r"Time Step: $T$", fontsize=30)
plt.ylabel(r"Regret: $R^L(T)$", fontsize=30)
plt.ylim(*ylim)
if save_figs:
    plt.savefig(input_dir + "/cumulative_regret.pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

# Regret comparison table

In [ ]:
T = 2000
regret = {}

algo_order = [
    "LGPUCB",
    "GPRegressor",
    "EmpiricalMean",
    "LogisticUCB1",
]

input_paths = {
    "ackley": "data/logistic_feedback/ackley",
    "branin": "data/logistic_feedback/branin",
    "eggholder": "data/logistic_feedback/eggholder",
    "hoelder": "data/logistic_feedback/hoelder",
    "matyas": "data/logistic_feedback/matyas",
    "michalewicz": "data/logistic_feedback/michalewicz",
    "rosenbrock": "data/logistic_feedback/rosenbrock",
}

for utility_name, input_dir in input_paths.items():
    for algo_name in algo_order:
        try:
            with open(input_dir + f"/{algo_name}.pkl", "rb") as f:
                data = pickle.load(f)
            if data["regret"].shape[0] != 20:
                print(f"Data shape mismatch for {utility_name} and {algo_name}")
            avg_cumulative_regret = jnp.cumsum(data["regret"], axis=-1)[:, T-1]
            mean = jnp.mean(avg_cumulative_regret)
            std_err = jnp.std(avg_cumulative_regret)/jnp.sqrt(avg_cumulative_regret.shape[0])
            regret[(utility_name, algo_name)] = (mean, std_err)
        except FileNotFoundError:
            print(f"File not found: {input_dir}/{algo_name}.pkl")
            regret[(utility_name, algo_name)] = (jnp.nan, jnp.nan)
df = pd.DataFrame(regret).T
df.columns = ["Mean", "StdErr"]
display(df["Mean"].unstack()[algo_order])
display(df["StdErr"].unstack()[algo_order])